In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import plotly.graph_objects as go
import json

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}

def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df['geometry'] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis='columns'), geometry='geometry')

In [2]:
vage_df = pd.read_csv('data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle.csv')

# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('SarriÃ -St. Gervasi', 'Sarrià-Sant Gervasi')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')

# vage_df.to_csv('data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle.csv', index=False)

In [3]:
total_vehicles_per_district = vage_df.groupby(['Nom_Districte']).Nombre.sum().reset_index()
total_vehicles_per_district.columns = ['Nom_Districte', 'Total_Vehicles']

old_vehicles_per_district = vage_df[vage_df.Antiguitat == 'MÃ©s de 20 anys'].groupby(['Nom_Districte']).Nombre.sum().reset_index()
old_vehicles_per_district.columns = ['Nom_Districte', 'Vehicles_20_Any']

In [4]:
merged = total_vehicles_per_district.merge(old_vehicles_per_district, on='Nom_Districte', how='left')
merged['Percentage'] = (merged['Vehicles_20_Any'] / merged['Total_Vehicles']) * 100
merged['Percentage'] = merged['Percentage'].map('{:,.2f}'.format)
merged = merged[merged.Nom_Districte != 'No consta']
merged['Percentage'] = merged['Percentage'].astype(float)

In [5]:
district_df = pd.read_csv("./data/district_zone/BarcelonaCiutat_Districtes.csv")

gdf = convert_wkt_to_geometry(district_df, 'geometria_wgs84')
gdf = gdf.rename(columns={'nom_districte': 'Nom_Districte'})

gdf_merged = gdf.merge(merged, on='Nom_Districte', how='left')

In [6]:
# fig = px.choropleth_mapbox(
#     gdf_merged,
#     geojson=gdf_merged,
#     locations=gdf_merged.index,
#     color="Percentage",
#     color_continuous_scale="OrRd",
#     mapbox_style="carto-positron",
#     center=CENTER_BARCELONA,
#     zoom=10,
#     hover_name="Nom_Districte",
#     hover_data={
#         "Percentage": True,
#     },
#     title="Percentage of vehicles older than 20 years per district",
# )

# fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

# fig.show()

In [ ]:
gdf_merged = gdf.merge(merged, on="Nom_Districte", how="left")

geojson = json.loads(gdf_merged.to_json())

fig = go.Figure(go.Choroplethmapbox(
    geojson=geojson,
    locations=gdf_merged.index,
    z=gdf_merged["Percentage"].astype(float),  
    colorscale="OrRd", 
    marker_opacity=0.7,
    marker_line_width=0.5,
    colorbar_title="Percentage (%)",
    text=gdf_merged["Nom_Districte"] + "<br>Percentage: " + gdf_merged["Percentage"].astype(str) + "%",
))

fig.update_layout(
    title_text="Percentage of vehicles older than 20 years per district",
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center=CENTER_BARCELONA,
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()

---

In [8]:
pop_df = pd.read_csv('data/population/2023/2023_pad_mdbas.csv')

# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('SarriÃ -Sant Gervasi', 'Sarrià-Sant Gervasi')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')

# pop_df.to_csv('data/population/2023/2023_pad_mdbas.csv', index=False)

In [9]:
pop_per_district = pop_df.groupby(['Nom_Districte']).Valor.sum().reset_index()
pop_per_district.columns = ['Nom_Districte', 'Population']

In [10]:
vtype_df = pd.read_csv("data/type_of_vehicle/2023/2023_Parc_vehicles_tipus_propulsio.csv")

# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('SarriÃ -St. Gervasi', 'Sarrià-Sant Gervasi')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')
# vtype_df['Tipus_Propulsio'] = vtype_df['Tipus_Propulsio'].replace('ElÃ¨ctrica', 'Elèctrica')
# df['Tipus_Propulsio'] = vtype_df['Tipus_Propulsio'].replace('HÃ­brid', 'Híbrid')

# vtype_df.to_csv('data/type_of_vehicle/2023/2023_Parc_vehicles_tipus_propulsio.csv', index=False)

In [11]:
vehuicles_per_district = vtype_df.groupby(['Nom_Districte']).Nombre.sum().reset_index()
vehuicles_per_district.columns = ['Nom_Districte', 'Total_Vehicles']

In [12]:
merged = vehuicles_per_district.merge(pop_per_district, on='Nom_Districte', how='left')
merged['Vehicles_Per_1000'] = (merged['Total_Vehicles'] / merged['Population']) * 1000
merged = merged[merged.Nom_Districte != 'No consta']
merged['Vehicles_Per_1000'] = merged['Vehicles_Per_1000'].map('{:,.2f}'.format)

In [ ]:
gdf_merged = gdf.merge(merged, on="Nom_Districte", how="left")

geojson = json.loads(gdf_merged.to_json())

fig = go.Figure(go.Choroplethmapbox(
    geojson=geojson,
    locations=gdf_merged.index,
    z=gdf_merged["Vehicles_Per_1000"].astype(float),  
    colorscale="OrRd", 
    marker_opacity=0.7,
    marker_line_width=0.5,
    colorbar_title="Vehicles Per 1000 habitants (‰)",
    text=gdf_merged["Nom_Districte"] + "<br>Vehicles Per 1000: " + gdf_merged["Vehicles_Per_1000"].astype(str) + "‰",
))

fig.update_layout(
    title_text="Vehicles Per 1000 habitant per district",
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center=CENTER_BARCELONA,
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()

---

In [14]:
# number of electric or hybrid vehicles per district
green_vehicles_per_district = vtype_df[(vtype_df.Tipus_Propulsio == 'Elèctrica') | (vtype_df.Tipus_Propulsio == 'Híbrid')].groupby(['Nom_Districte']).Nombre.sum().reset_index()
green_vehicles_per_district.columns = ['Nom_Districte', 'Green_Vehicles']

In [15]:
merged = vehuicles_per_district.merge(green_vehicles_per_district, on='Nom_Districte', how='left')
merged['Percentage'] = (merged['Green_Vehicles'] / merged['Total_Vehicles']) * 100
merged['Percentage'] = merged['Percentage'].map('{:,.2f}'.format)

In [ ]:
gdf_merged = gdf.merge(merged, on="Nom_Districte", how="left")

geojson = json.loads(gdf_merged.to_json())

fig = go.Figure(go.Choroplethmapbox(
    geojson=geojson,
    locations=gdf_merged.index,
    z=gdf_merged["Percentage"].astype(float),  
    colorscale="OrRd", 
    marker_opacity=0.7,
    marker_line_width=0.5,
    colorbar_title="Percentage (%)",
    text=gdf_merged["Nom_Districte"] + "<br>Percentage: " + gdf_merged["Percentage"].astype(str) + "%",
    reversescale=True
))

fig.update_layout(
    title_text="Percentage of green vehicles per district",
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center=CENTER_BARCELONA,
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()